In [2]:
import sys
import os
from PIL import Image
import cv2

In [28]:
focal_length = 800.0
scaling_factor = 2000.0


In [29]:
def generate_point_cloud(rgb_file, depth_file, ply_file):
    rgb = Image.open(rgb_file).convert("RGB")
    depth = Image.open(depth_file).convert("I")
    depth = depth.resize(rgb.size)

    centerX = rgb.size[1] / 2
    centerY = rgb.size[0] / 2

    if rgb.size != depth.size:
        raise Exception("Color and depth image do not have the same resolution.")
    if rgb.mode != "RGB":
        raise Exception("Color image is not in RGB format")
    if depth.mode != "I":
        raise Exception("Depth image is not in intensity format")


    points = []    
    for v in range(rgb.size[1]):
        for u in range(rgb.size[0]):
            color = rgb.getpixel((u,v))
            Z = depth.getpixel((u,v)) / scaling_factor
            if Z==0: continue
            X = (u - centerX) * Z / focal_length
            Y = (v - centerY) * Z / focal_length
            points.append("%f %f %f %d %d %d 0\n"%(X,Y,Z,color[0],color[1],color[2]))
    file = open(ply_file,"w")
    file.write('''ply
format ascii 1.0
element vertex %d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
property uchar alpha
end_header
%s
'''%(len(points),"".join(points)))
    file.close()
    

In [30]:
rgb_file = "man_color.png"
depth_file = "man_depth.png"
ply_file = "result.ply"

In [31]:
generate_point_cloud(rgb_file, depth_file, ply_file)